In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
cd ..

/Users/Shian/Programs/Python/Statistical-Machine-Learning-Project


In [3]:
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

Using Theano backend.


In [4]:
with open('data/train.csv', 'rb') as csvfile:
    training_data = np.loadtxt(csvfile, delimiter=',')

In [5]:
perm_data = np.random.permutation(training_data)
n_samples = len(perm_data)

img_rows, img_cols = 33, 13

data = perm_data[:, 9:439].reshape(n_samples, 1, img_rows, img_cols)
target = perm_data[:, 1].astype(int)

input_shape = (1, img_rows, img_cols)

In [6]:
nb_classes = 98

# input image dimensions
img_rows, img_cols = 33, 13
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

X_data = data.astype('float32')
X_data /= 255

In [31]:
# Random image generator
datagen = ImageDataGenerator(
    rotation_range=15,
    shear_range=0.1,
    zoom_range=0.05)

In [11]:
# convert class vectors to binary class matrices
Y_target = np_utils.to_categorical(target, nb_classes)

In [47]:
# Model parameters
batch_size = 128
nb_filters = 32
nb_epoch = 200

# Construct model, structure similar to LeNet5 from MNIST author
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Adadelta is some kind of adaptive learning rate optimiser, was just in the code I copied
model.compile(loss='categorical_crossentropy',
              optimizer='Adadelta',
              metrics=['accuracy'])

In [48]:
# Model fit from data served from the generator in batches of 128, validated on another random 5000 or so 
# observations served from generator.
model.fit_generator(generator = datagen.flow(X_data, Y_target, batch_size=batch_size),
                    samples_per_epoch = len(X_data),
                    nb_epoch = nb_epoch,
                    validation_data = datagen.flow(X_data, Y_target, batch_size=int(len(X_data)/10)),
                    nb_val_samples = 1,
                    verbose = 1)

Epoch 1/200
50282/50282 [==============================] - 80s - loss: 2.5349 - acc: 0.4154 - val_loss: 1.9920 - val_acc: 0.5672
Epoch 2/200
50282/50282 [==============================] - 76s - loss: 2.1109 - acc: 0.5503 - val_loss: 1.8364 - val_acc: 0.6185
Epoch 3/200
50282/50282 [==============================] - 77s - loss: 2.0096 - acc: 0.5774 - val_loss: 1.7946 - val_acc: 0.6207
Epoch 4/200
50282/50282 [==============================] - 76s - loss: 1.9589 - acc: 0.5918 - val_loss: 1.7577 - val_acc: 0.6249
Epoch 5/200
50282/50282 [==============================] - 77s - loss: 1.9188 - acc: 0.6043 - val_loss: 1.7620 - val_acc: 0.6279
Epoch 6/200
50282/50282 [==============================] - 74s - loss: 1.8865 - acc: 0.6117 - val_loss: 2.0962 - val_acc: 0.5000
Epoch 7/200
50282/50282 [==============================] - 77s - loss: 1.8627 - acc: 0.6197 - val_loss: 1.6752 - val_acc: 0.6484
Epoch 8/200
50282/50282 [==============================] - 77s - loss: 1.8479 - acc: 0.6228 - val

In [50]:
with open('data/test.csv', 'rb') as csvfile:
    testing_data = np.loadtxt(csvfile, delimiter=',')

In [57]:
test_bitmaps = testing_data[:, 9:439].reshape(testing_data.shape[0], 1, img_rows, img_cols)
test_bitmaps /= 255
test_ids = testing_data[:, 0]
test_bitmaps

In [58]:
test_predict = np_utils.categorical_probas_to_classes(model.predict(test_bitmaps))

In [60]:
output = np.column_stack((test_ids, test_predict))

In [63]:
save_path = os.path.join('Convolutional Neural Network', 'kerasCNN_Preprocessed.csv')
np.savetxt('kerasCNN_Preprocessed.csv', output, fmt='%d', header="Id,Character", delimiter=",", comments="")

In [62]:
model_path = os.path.join('Convolutional Neural Network', 'model.h5')
model.save(model_path)